In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="pipeline")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: 20220117_ws_pia
Azure region: brazilsouth
Subscription id: 10fb68cc-c4bb-48ac-991a-841331d107e9
Resource group: ia


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

cpu_cluster_name = 'Cluster'
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print("Found existing cluster, using it....")
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size="Standard_D2_V2", max_nodes=4)
    cpu_cluster = ComputeTarget.create(workspace=ws, name=cpu_cluster_name, provisioning_configuration=compute_config)

cpu_cluster.wait_for_completion(show_output=True)


Found existing cluster, using it....
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [3]:
from azureml.widgets import RunDetails
from azureml.core import ScriptRunConfig
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
import os
from azureml.core import Environment

sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

# Specify parameter sampler
ps = RandomParameterSampling({
        "--C":uniform(0.01,1),
        "--max_iter": choice(50, 80, 100)})

# Specify a Policy
policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
est = ScriptRunConfig(source_directory=".",
                      compute_target=cpu_cluster,
                      script="train.py",
                      environment=sklearn_env)

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(run_config=est,
                                     hyperparameter_sampling=ps,
                                     primary_metric_name='Accuracy',
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                     max_concurrent_runs=4,
                                     policy=policy,
                                     max_total_runs=10)   

No Python version provided, defaulting to "3.8.13"


In [4]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

hyperdrive_run = exp.submit(config=hyperdrive_config)
RunDetails(hyperdrive_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [5]:
hyperdrive_run.wait_for_completion(show_output=True)

RunId: HD_4cb29731-4fc4-4172-bb07-47dae9e334ae
Web View: https://ml.azure.com/runs/HD_4cb29731-4fc4-4172-bb07-47dae9e334ae?wsid=/subscriptions/10fb68cc-c4bb-48ac-991a-841331d107e9/resourcegroups/ia/workspaces/20220117_ws_pia&tid=e8f5849e-5b15-4c2f-b4f3-0386b604971e

Streaming azureml-logs/hyperdrive.txt

[2022-10-31T06:24:21.376860][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space
[2022-10-31T06:24:21.8490575Z][SCHEDULER][INFO]Scheduling job, id='HD_4cb29731-4fc4-4172-bb07-47dae9e334ae_1' 
[2022-10-31T06:24:21.9125844Z][SCHEDULER][INFO]Scheduling job, id='HD_4cb29731-4fc4-4172-bb07-47dae9e334ae_2' 
[2022-10-31T06:24:22.0209143Z][SCHEDULER][INFO]Scheduling job, id='HD_4cb29731-4fc4-4172-bb07-47dae9e334ae_3' 
[2022-10-31T06:24:21.974931][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.
[2022-10-31T06:24:21.6922227Z][SCHEDULER][INFO]Scheduling job, id='HD_4cb29731-4fc4-4172-bb07-47dae9e334ae_0' 
[2022-10-31T06:24:22

{'runId': 'HD_4cb29731-4fc4-4172-bb07-47dae9e334ae',
 'target': 'Cluster',
 'status': 'Completed',
 'startTimeUtc': '2022-10-31T06:24:19.933835Z',
 'endTimeUtc': '2022-10-31T06:42:28.224655Z',
 'services': {},
 'properties': {'primary_metric_config': '{"name":"Accuracy","goal":"maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': 'afb62bf8-2a20-4845-a90b-8186b80ddca9',
  'user_agent': 'python/3.8.5 (Linux-5.15.0-1017-azure-x86_64-with-glibc2.10) msrest/0.7.1 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.44.0',
  'space_size': 'infinite_space_size',
  'score': '0.9118079823432039',
  'best_child_run_id': 'HD_4cb29731-4fc4-4172-bb07-47dae9e334ae_0',
  'best_metric_status': 'Succeeded',
  'best_data_container_id': 'dcid.HD_4cb29731-4fc4-4172-bb07-47dae9e334ae_0'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'configuration': None,
  'attribution': None,
  'telemetryValues':

In [7]:
import joblib
# Get your best run and save the model from that run.

best_run = hyperdrive_run.get_best_run_by_primary_metric()

best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()['runDefinition']['arguments']

print('\n Best Run Id: ', best_run.id)
print('\n Accuracy:', best_run_metrics['Accuracy'])
print('\n parameter_values :',parameter_values)


 Best Run Id:  HD_4cb29731-4fc4-4172-bb07-47dae9e334ae_0

 Accuracy: 0.9118079823432039

 parameter_values : ['--C', '0.5614639281996592', '--max_iter', '80']


In [8]:
best_run

Experiment,Id,Type,Status,Details Page,Docs Page
pipeline,HD_4cb29731-4fc4-4172-bb07-47dae9e334ae_0,azureml.scriptrun,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [9]:
#create path to save best run id
os.makedirs('outputs', exist_ok=True)

#save best run id
joblib.dump(value= best_run.id, filename = 'outputs/model.joblib')

['outputs/model.joblib']

In [10]:
best_run.get_file_names()

['azureml-logs/20_image_build_log.txt',
 'logs/azureml/dataprep/0/backgroundProcess.log',
 'logs/azureml/dataprep/0/backgroundProcess_Telemetry.log',
 'logs/azureml/dataprep/0/rslex.log.2022-10-31-06',
 'system_logs/cs_capability/cs-capability.log',
 'system_logs/hosttools_capability/hosttools-capability.log',
 'system_logs/lifecycler/execution-wrapper.log',
 'system_logs/lifecycler/lifecycler.log',
 'system_logs/metrics_capability/metrics-capability.log',
 'system_logs/snapshot_capability/snapshot-capability.log',
 'user_logs/std_log.txt']

## To run with Auto ML:

In [11]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###

df = TabularDatasetFactory.from_delimited_files("https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv")

In [13]:
from train import clean_data
from sklearn.model_selection import train_test_split
from azureml.core import Dataset
import pandas as pd

# Use the clean_data function to clean your data.
x, y = clean_data(df)

x_train,x_test,y_train,y_test = train_test_split(x, y, test_size=0.33,random_state=42)

final_train = pd.concat([x_train, y_train], axis=1)

In [14]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    training_data=final_train,
    label_column_name='y',
    n_cross_validations=4)

In [15]:
# Submit your automl run

exp = Experiment(workspace=ws, name="automl-model")
automl_run = exp.submit(automl_config, show_output=True)

No run_configuration provided, running on local with default configuration
Running in the active local environment.


Experiment,Id,Type,Status,Details Page,Docs Page
automl-model,AutoML_a41235a5-512b-4f45-83a0-263c466acbfe,automl,Preparing,Link to Azure Machine Learning studio,Link to Documentation


Current status: DatasetEvaluation. Gathering dataset statistics.
Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetFeaturizationCompleted. Completed fit featurizers and featurizing the dataset.
Current status: DatasetBalancing. Performing class balancing sweeping
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.

********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one cl

2022-10-31:07:25:20,664 INFO     [explanation_client.py:334] Using default datastore for uploads


Current status: EngineeredFeatureExplanations. Computation of engineered features completed
Current status: RawFeaturesExplanations. Computation of raw features started
Current status: RawFeaturesExplanations. Computation of raw features completed
Current status: BestRunExplainModel. Best run model explanations completed
********************************************************************************************


In [16]:
RunDetails(automl_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

In [19]:
best_automl_run, fitted_model = automl_run.get_output()

In [20]:
best_automl_run

Experiment,Id,Type,Status,Details Page,Docs Page
automl-model,AutoML_a41235a5-512b-4f45-83a0-263c466acbfe_31,,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [21]:
fitted_model

Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=False, enable_feature_sweeping=True, feature_sweeping_config={}, feature_sweeping_timeout=86400, featurization_config=None, force_text_dnn=False, is_cross_validation=True, is_onnx_compatible=False, observer=None, task='classification', working_dir='/mnt/batch/tasks/shared/LS_root/mount...
                 PreFittedSoftVotingClassifier(classification_labels=array([0, 1]), estimators=[('24', Pipeline(memory=None, steps=[('standardscalerwrapper', StandardScalerWrapper(copy=True, with_mean=False, with_std=False)), ('xgboostclassifier', XGBoostClassifier(booster='gbtree', colsample_bytree=1, eta=0.05, gamma=0, max_depth=6, max_leaves=0, n_estimators=200, n_jobs=1, objective='reg:logistic', problem_info=ProblemInfo(gpu_training_param_dict={'processing_unit_type': 'cpu'}), random_state=0, reg_alpha=0.625, reg_lambda=0.8333333333333334, subsample=0.8, tree_method='auto'))], verbose=False)), ('

In [22]:
best_automl_run.id

'AutoML_a41235a5-512b-4f45-83a0-263c466acbfe_31'

In [23]:
#create path to save best run id
os.makedirs('outputs', exist_ok=True)

#save best run id
joblib.dump(value= best_automl_run.id, filename = 'outputs/automl_model.joblib')

['outputs/automl_model.joblib']

In [24]:
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print("Found existing cluster, deleting it....")
    cpu_cluster.delete()
except ComputeTargetException:
    print("Cluster not found.")

Found existing cluster, deleting it....
